# Test Fundational Models

Test de modelos fundacionales con los datos del time-300b

In [1]:
import random
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from Time_MoE.time_moe.datasets.time_moe_dataset import TimeMoEDataset
from sklearn.model_selection import train_test_split
# Importing custom functions
import sys
import os
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_path)

from baseline.functions import load_data,create_intervals,create_windows,smape,smape_chunked,sample_fraction
from sklearn.metrics import mean_absolute_error, mean_squared_error

def load_data_clean():
    ds = TimeMoEDataset(data_folder='Time-300B\healthcare',normalization_method='zero')

    verbose = True
    total = len(ds)
    valid_indices = []
    # Iterar y filtrar
    for i in range(total):
        try:
            seq = ds[i]  # seq es numpy.ndarray según comprobaste
        except Exception as e:
            # Si hay error al obtener la secuencia, lo avisamos y saltamos
            if verbose:
                print(f"Advertencia: no se pudo obtener ds[{i}]: {e}")
            continue
        
        # Comprobación: si todos los valores son NaN, lo descartamos
        # seq es numpy.ndarray; cuidado si dims especiales, pero np.isnan funcionará elementwise.
        try:
            if not np.all(np.isnan(seq)):
                valid_indices.append(i)
        except Exception as e:
            # En caso de que seq no sea array puro, convertir primero:
            try:
                arr = np.array(seq)
                if not np.all(np.isnan(arr)):
                    valid_indices.append(i)
            except Exception as e2:
                if verbose:
                    print(f"Error al verificar NaN en secuencia índice {i}: {e2}")
                # Decidir si incluirla o no. Aquí optamos por descartarla:
                continue
    
    valid_count = len(valid_indices)
    if verbose:
        print(f"Secuencias totales en ds: {total}")
        print(f"Secuencias válidas (no todo NaN): {valid_count}")
        print(f"Secuencias descartadas: {total - valid_count}")
        sequences_validas = []

    for idx in valid_indices:
        try:
            sequences_validas.append(ds[idx])
        except Exception as e:
            if verbose:
                print(f"Error al extraer ds[{idx}] después de filtrar: {e}")
            # Podrías decidir saltar o detener. Aquí solo saltamos.
    return sequences_validas

def create_windows_from_sequences(sequences, window_size=15, horizon=1):
    """
    Dada una lista de secuencias (numpy arrays 1D), crea ventanas deslizantes:
    - X: array de shape (num_samples, window_size, 1)
    - y: array de shape (num_samples,)
    Cada muestra usa window_size pasos para predecir el siguiente valor (horizon=1).
    """
    X_list = []
    y_list = []
    for seq in sequences:
        # Asegurar numpy array
        arr = np.array(seq).astype(float)
        T = arr.shape[0]
        # Solo si la longitud es mayor que window_size + horizon - 1
        if T >= window_size + horizon:
            for start in range(0, T - window_size - horizon + 1):
                window = arr[start:start+window_size]
                target = arr[start+window_size:start+window_size+horizon]
                # Para horizon=1, target es un array de longitud 1; tomamos el escalar
                X_list.append(window.reshape(window_size, 1))
                y_list.append(target[0] if horizon == 1 else target)
    if len(X_list) == 0:
        return np.empty((0, window_size, 1)), np.empty((0,))
    X = np.stack(X_list, axis=0)
    y = np.array(y_list)

    # Supongamos X tiene forma (N, window_size, 1), y y forma (N,)
    mask_valid = ~np.isnan(X).any(axis=(1,2)) & ~np.isnan(y)
    # Mantener solo muestras sin NaN:
    X_clean = X[mask_valid]
    y_clean = y[mask_valid]
    print("De", X.shape[0], "muestras, quedan", X_clean.shape[0], "sin NaN")

    return X_clean, y_clean

In [2]:
ds = load_data_clean()

X, y = create_windows_from_sequences(ds, window_size=15, horizon=1)

Secuencias totales en ds: 1752
Secuencias válidas (no todo NaN): 1752
Secuencias descartadas: 0
De 433317 muestras, quedan 433317 sin NaN


In [3]:
# Load time_moe
X_3, y_3   = sample_fraction(X, y, 0.03, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_3, y_3, test_size=0.2, shuffle=True, random_state=42)

### Times-MoE

In [5]:
# Teacher prediction using Time-MoE
from transformers import AutoModelForCausalLM
import torch
import numpy as np    

model = AutoModelForCausalLM.from_pretrained(
    'Maple728/TimeMoE-50M',
    device_map="cpu",  # use "cpu" for CPU inference, and "cuda" for GPU inference.
    trust_remote_code=True,
)

train_predict_teacher = []
val_predict_teacher = []

for arr in X_val:                   
    t = torch.from_numpy(arr.reshape(1, 15)).float()
    mean = t.mean(dim=-1, keepdim=True)   # (1,1)
    std  = t.std(dim=-1, keepdim=True)    # (1,1)
    std = std.clamp(min=1e-6) # Evitar división por cero
    normed_seq = (t - mean) / std         # (1,15)
    output = model.generate(normed_seq, max_new_tokens=1)  
    normed_pred = output[:, -1:]            # (1,1)
    pred = normed_pred * std + mean         # (1,1)
    val_predict_teacher.append(pred.item())  # Guardar las predicciones desnormalizadas

In [6]:
#Resultados de modelos teacher sobre el conjunto de validación 
mae = mean_absolute_error(y_val, val_predict_teacher)
mse = mean_squared_error(y_val, val_predict_teacher)
smape_val = smape(y_val, val_predict_teacher)

print("MAE:", mae)
print("MSE:", mse)
print("SMAPE:", smape_val)

MAE: 0.29947147856463113
MSE: 0.2808381897192965
SMAPE: 30.35469758164513


In [4]:
from samay.dataset import MoiraiDataset
from samay.model import MoiraiTSModel

repo = "Salesforce/moirai-moe-1.0-R-small"
config = {
        "context_len": 128,
        "horizon_len": 64,
        "num_layers": 100,
        "model_type": "moirai-moe",
        "model_size": "small"
    }

moirai_model = MoiraiTSModel(repo=repo, config=config)

In [12]:
X_val.shape

(2600, 15, 1)

In [13]:
# 1) Configuración de tu dataset de test
test_dataset = MoiraiDataset(
    name="ett",
    mode="test",
    path="X_val.csv",
    datetime_col="date",
    freq="h",
    context_len=config['context_len'],
    horizon_len=config['horizon_len']
)


Normalizing the dataset


In [16]:
eval_results, trues, preds, histories = moirai_model.evaluate(test_dataset, metrics=["MSE", "MASE"])

Forecasting done....now testing


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed